# 🎓 Campus Tour 3D Model Generator (MVP Mode)

**Convert campus videos to 3D Interactive Walkthroughs**

## 🚀 Quick Start (MVP Mode)

1. Upload video to Google Drive
2. Set `RUN_MVP = True` below
3. Click **Runtime → Run All**
4. Wait for download link
5. Upload to GitHub → Done!

---

## 📹 Video Requirements

| Setting | Recommended |
|---------|-------------|
| Max Length | 30-60 seconds |
| Resolution | 1080p |
| Frame Rate | 30fps |
| Motion | Slow walk/orbit |
| Lighting | Consistent (golden hour) |

In [ ]:
# @title ## ⚡ MVP Configuration (SET THIS!)

# @markdown ### 🎯 MVP Mode - Fire and Forget
RUN_MVP = True  # @param {type:"boolean"}

# @markdown ### 📹 Video Path on Google Drive
VIDEO_PATH = "/content/drive/MyDrive/campus/tour.mp4"  # @param {type:"string"}

# @markdown ### 🏷️ Building Name (no spaces)
OUTPUT_NAME = "cs_building"  # @param {type:"string"}

# @markdown ### 🎨 Quality (higher = slower)
QUALITY = "medium"  # @param ["low", "medium", "high"]

# @markdown ### 🎥 Camera Path Type
CAMERA_PATH = "circular"  # @param ["circular", "forward", "figure8"]

In [ ]:
# @title ## 🔧 Quality Settings (Auto-configured)
import os

if RUN_MVP:
    # Auto-config based on quality setting
    quality_config = {
        "low": {"iterations": 8000, "downscale": 3, "num_frames": 100},
        "medium": {"iterations": 15000, "downscale": 2, "num_frames": 200},
        "high": {"iterations": 25000, "downscale": 1, "num_frames": 300}
    }
    
    ITERATIONS = quality_config[QUALITY]["iterations"]
    DOWNSCALE = quality_config[QUALITY]["downscale"]
    NUM_FRAMES = quality_config[QUALITY]["num_frames"]
else:
    ITERATIONS = 15000
    DOWNSCALE = 2
    NUM_FRAMES = 200

print(f"🎯 Running with:")
print(f"  - Iterations: {ITERATIONS}")
print(f"  - Downscale: {DOWNSCALE}x")
print(f"  - Max frames: {NUM_FRAMES}")
print(f"  - Camera path: {CAMERA_PATH}")

In [ ]:
# @title ## 🚀 Step 1: Install Dependencies
!pip install -q nerfstudio
!pip install -q gsplat

In [ ]:
# @title ## 📂 Step 2: Setup Folders

BASE_DIR = "/content/campus_output"
DATA_DIR = os.path.join(BASE_DIR, "data", OUTPUT_NAME)
OUTPUT_DIR = os.path.join(BASE_DIR, "output", OUTPUT_NAME)

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Data: {DATA_DIR}")
print(f"📁 Output: {OUTPUT_DIR}")

In [ ]:
# @title ## 📥 Step 3: Copy & Optimize Video
import shutil

if os.path.exists(VIDEO_PATH):
    local_video = os.path.join(DATA_DIR, "input.mp4")
    print(f"📥 Copying video...")
    shutil.copy2(VIDEO_PATH, local_video)
    
    # Get video info
    import subprocess
    result = subprocess.run([
        "ffprobe", "-v", "error", "-select_streams", "v:0",
        "-show_entries", "stream=width,height,duration",
        "-of", "json", local_video
    ], capture_output=True, text=True)
    
    print(f"✅ Video ready: {os.path.getsize(local_video) / (1024*1024):.1f} MB")
else:
    print(f"❌ Video not found: {VIDEO_PATH}")
    print("\n📝 Please upload your video to Google Drive and update VIDEO_PATH")

In [ ]:
# @title ## 🎨 Step 4: Process Video
import subprocess

cmd = [
    "ns-process-data",
    "video",
    "--data", DATA_DIR,
    "--output-dir", DATA_DIR,
    "--matching-method", "exhaustive",
    "--num-downscale", str(DOWNSCALE),
    "--max-frames", str(NUM_FRAMES)
]

print("🎨 Processing video...")
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout[-1000:] if result.stdout else "Done!")

In [ ]:
# @title ## 🧠 Step 5: Train Model

cmd = [
    "ns-train",
    "gaussian-splatting",
    "--data", DATA_DIR,
    "--output-dir", OUTPUT_DIR,
    "--max-num-iterations", str(ITERATIONS),
    "--pipeline.model.coarse-to-fine",
    "--viewer.enabled", "False"
]

print(f"🧠 Training ({ITERATIONS} iterations)...")
print(f"⏱️ Estimated: {ITERATIONS // 600} minutes\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print("✅ Training complete!")

In [ ]:
# @title ## 📦 Step 6: Export 3D Model

# Find config
config_path = None
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        if f.endswith('.yml'):
            config_path = os.path.join(root, f)
            break

if config_path:
    export_dir = os.path.join(OUTPUT_DIR, "exports")
    os.makedirs(export_dir, exist_ok=True)
    
    cmd = [
        "ns-export",
        "gaussian-splatting",
        "--load-config", config_path,
        "--output-dir", export_dir,
        "--export-format", "ply"
    ]
    
    print("📦 Exporting to .ply...")
    subprocess.run(cmd, capture_output=True)
    
    ply_files = [f for f in os.listdir(export_dir) if f.endswith('.ply')]
    if ply_files:
        for f in ply_files:
            size = os.path.getsize(os.path.join(export_dir, f)) / (1024*1024)
            print(f"  ✅ {f} ({size:.1f} MB)")

In [ ]:
# @title ## 🎥 Step 7: Generate Camera Path

# Create a simple camera path JSON
import json

camera_paths = {
    "circular": [
        {"position": [8, 2, 8], "lookAt": [0, 0, 0], "duration": 5000},
        {"position": [8, 2, -8], "lookAt": [0, 0, 0], "duration": 5000},
        {"position": [-8, 2, -8], "lookAt": [0, 0, 0], "duration": 5000},
        {"position": [-8, 2, 8], "lookAt": [0, 0, 0], "duration": 5000},
    ],
    "forward": [
        {"position": [0, 2, 10], "lookAt": [0, 0, 0], "duration": 8000},
        {"position": [0, 2, 5], "lookAt": [0, 0, 0], "duration": 8000},
        {"position": [0, 2, 0], "lookAt": [0, 0, 0], "duration": 8000},
    ],
    "figure8": [
        {"position": [8, 2, 0], "lookAt": [0, 0, 0], "duration": 4000},
        {"position": [0, 2, 8], "lookAt": [0, 0, 0], "duration": 4000},
        {"position": [-8, 2, 0], "lookAt": [0, 0, 0], "duration": 4000},
        {"position": [0, 2, -8], "lookAt": [0, 0, 0], "duration": 4000},
    ]
}

path_data = camera_paths.get(CAMERA_PATH, camera_paths["circular"])

# Save to file
path_file = os.path.join(OUTPUT_DIR, "exports", "camera_path.json")
with open(path_file, 'w') as f:
    json.dump(path_data, f, indent=2)

print(f"✅ Camera path saved: {CAMERA_PATH}")
print(f"📄 {path_file}")

In [ ]:
# @title ## ⬇️ Step 8: Download

from google.colab import files

export_dir = os.path.join(OUTPUT_DIR, "exports")
ply_files = [f for f in os.listdir(export_dir) if f.endswith('.ply')]

print("📦 Files ready for download:\n")
for f in ply_files:
    filepath = os.path.join(export_dir, f)
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"  📄 {f} ({size:.1f} MB)")

print("\n⬇️ Click to download:")
for f in ply_files:
    files.download(os.path.join(export_dir, f))

print("\n✅ Done! Upload the .ply file to GitHub → https://github.com/rajpal07/campus-assets/tree/main/buildings")

---

## 📋 How to Use the 3D Model

1. **Download** the .ply file above
2. **Upload** to GitHub: https://github.com/rajpal07/campus-assets/tree/main/buildings
3. **Create Release**: https://github.com/rajpal07/campus-assets/releases/new
4. **The app** will automatically load your 3D model!

---

## 💡 Tips for Great Results

- **Slow motion**: Walk slowly around the building
- **Good light**: Golden hour (sunset) works best
- **Steady camera**: Use a gimbal or lean against walls
- **Full coverage**: Capture all sides of the building